STOCK MARKET VOLATILITY AND LEARNING
Adam, Marcet, Nicolini (2006-)

MAIN PROGRAM to:
- Load the data and compute sample moments (calls computeWeightMatrix)
- Set the options (model, estimate or not, which statistics to match, weighting matrix)
- Simulate (or estimate) Learning model (calls est_learning)
- Simulate (or estimate) Abel RA model (calls est_Abel) - can be ignored
- Show tables on the screen (calls show_table)

Code originally by Davide Debortoli, September 2007
Clearing and Comment by Tongbin Zhang, Feb, 2015
Python translation, 2025

In [8]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import minimize, fmin
from scipy.linalg import inv, eig
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from typing import Tuple, Optional, Dict, List
import warnings
import pickle
import json

warnings.filterwarnings('ignore')

In [9]:
# ============================================================================
# Configuration Classes
# ============================================================================

@dataclass
class ModelConfig:
    """Configuration for the learning model"""
    realizations: int = 1000
    CI: float = 0.95
    init: int = 2
    lags: int = 5
    PD_max: float = 500
    ratio_proj: float = 5/2.5
    method: str = 'tracking'  # 'OLS', 'tracking', or 'switching_weights'
    proj_type: int = 1
    C_DIV: int = 1  # 0 for baseline, 1 for C≠D
    
@dataclass
class Parameters:
    """Model parameters"""
    a: float = 1.00016      # Average dividend growth
    s: float = 0.02         # Std of delta ln(d)
    sigma: float = 5        # Risk aversion
    delta: float = 150      # Discount factor
    alpha: float = 163.4043 # Learning parameter

class GlobalData:
    """Container for global data"""
    def __init__(self):
        self.MOM_DATA = None
        self.N = None
        self.dS_Sw_dS = None
        self.realizations = 1000
        self.init = 2
        self.CI = 0.95
        self.ratio_proj = 5/2.5

In [10]:
# ============================================================================
# Data Loading and Preprocessing
# ============================================================================

def load_series_data(filepath: Optional[str] = None) -> Dict[str, np.ndarray]:
    """
    Load financial series data
    
    Args:
        filepath: Path to data file (if None, generates synthetic data)
    
    Returns:
        Dictionary containing PD, rs, rb, DivGrowth arrays
    """
    if filepath:
        # Load actual data from file
        # This would be implemented based on your data format
        data = pd.read_csv(filepath)
        return {
            'PD': data['PD'].values,
            'rs': data['rs'].values,
            'rb': data['rb'].values,
            'DivGrowth': data['DivGrowth'].values
        }
    else:
        # Generate synthetic data for demonstration
        np.random.seed(123)
        n_periods = 500
        
        PD = 100 + 20 * np.random.randn(n_periods).cumsum()
        rs = 1.02 + 0.15 * np.random.randn(n_periods)
        rb = 1.01 + 0.02 * np.random.randn(n_periods)
        DivGrowth = 1.002 + 0.03 * np.random.randn(n_periods)
        
        return {'PD': PD, 'rs': rs, 'rb': rb, 'DivGrowth': DivGrowth}

def compute_rho(data: Dict[str, np.ndarray]) -> Dict[str, float]:
    """
    Compute autocorrelation coefficients
    Equivalent to checkRho.m
    """
    PD = data['PD']
    
    # Create lagged product
    PD_PDlag = np.concatenate([[np.nan], PD[1:] * PD[:-1]])
    
    # Remove NaN and last 20 observations
    PD_PDlag = PD_PDlag[1:-20]
    PDlag = PD[:-21]
    PDlag2 = PDlag ** 2
    PD_clean = PD[1:-20]
    PD2 = PD_clean ** 2
    
    # Asymptotic approximation
    rho_approx = (np.mean(PD_PDlag) - np.mean(PD_clean)**2) / \
                 (np.mean(PD2) - np.mean(PD_clean)**2)
    
    # Exact sample correlation
    rho = (np.mean(PD_PDlag) - np.mean(PD_clean) * np.mean(PDlag)) / (
        np.sqrt(np.mean(PD2) - np.mean(PD_clean)**2) * 
        np.sqrt(np.mean(PDlag2) - np.mean(PDlag)**2)
    )
    
    return {
        'rho_approx': rho_approx,
        'rho_exact': rho,
        'corr': np.corrcoef(PD_clean, PDlag)[0, 1]
    }

In [11]:
# ============================================================================
# Weight Matrix Computation
# ============================================================================

def compute_weight_matrix(lag: int = 5, method: int = 1, 
                         do_fullsample: int = 0) -> Tuple[np.ndarray, np.ndarray, int]:
    """
    Compute moments and weighting matrix
    Equivalent to computeWeightMatrix.m
    """
    # Load data
    data = load_series_data()
    PD = data['PD']
    rs = data['rs']
    rb = data['rb']
    DivGrowth = data['DivGrowth']
    
    # Build additional series
    PD_PDlag = np.concatenate([[np.nan], PD[1:] * PD[:-1]])
    rs_rslag = np.concatenate([[np.nan], rs[1:] * rs[:-1]])
    
    # Create indices
    stock_index = np.cumprod(rs[1:])
    bond_index = np.cumprod(rb[1:])
    
    # Compute excess returns
    if len(stock_index) > 4*lag:
        excret = (stock_index[4*lag:] / stock_index[:-4*lag]) / \
                 (bond_index[4*lag:] / bond_index[:-4*lag])
    else:
        excret = np.array([])
    
    # Build h(y) vector - moments
    end_idx = len(rs) - 4*lag
    hy = np.array([
        rs[1:end_idx],
        rb[1:end_idx],
        PD[1:end_idx],
        DivGrowth[1:end_idx],
        rs[1:end_idx]**2,
        PD[1:end_idx]**2,
        DivGrowth[1:end_idx]**2,
        PD_PDlag[1:end_idx],
        np.pad(excret, (0, max(0, end_idx-1-len(excret))), 'constant'),
        np.pad(excret**2, (0, max(0, end_idx-1-len(excret))), 'constant'),
        np.pad(excret * PD[:len(excret)], (0, max(0, end_idx-1-len(excret))), 'constant'),
        rb[1:end_idx]**2,
        rs_rslag[1:end_idx]
    ])
    
    N = hy.shape[1]
    M = np.mean(hy, axis=1)
    
    # Compute statistics vector S
    S = compute_statistics_vector(M)
    
    # Compute derivatives
    dS = compute_derivatives(S, M)
    
    # Compute Newey-West covariance
    Sw = compute_newey_west(hy, M, N, method)
    
    # Weight matrix
    dS_Sw_dS = dS @ Sw @ dS.T
    
    return dS_Sw_dS, S, N

def compute_statistics_vector(M: np.ndarray) -> np.ndarray:
    """Compute statistics vector from moments"""
    S = np.zeros(12)
    S[0:4] = M[0:4]
    S[4] = np.sqrt(max(0, M[4] - M[0]**2))     # sigma_rs
    S[5] = np.sqrt(max(0, M[5] - M[2]**2))     # sigma_PD
    S[6] = np.sqrt(max(0, M[6] - M[3]**2))     # sigma_D/d
    S[7] = (M[7] - M[2]**2) / S[5]**2 if S[5] > 0 else 0  # autocorr_PD
    S[8] = (M[10] - M[2]*M[8]) / S[5]**2 if S[5] > 0 else 0  # betaPD
    S[9] = S[8]**2 * S[5]**2 / (M[9] - M[8]**2) if (M[9] - M[8]**2) > 0 else 0  # R^2
    S[10] = np.sqrt(max(0, M[11] - M[1]**2))   # sigma_rb
    S[11] = (M[12] - M[0]**2) / S[4]**2 if S[4] > 0 else 0  # autocorr_rs
    return S

def compute_derivatives(S: np.ndarray, M: np.ndarray) -> np.ndarray:
    """Compute derivatives dS/dM"""
    dS = np.zeros((len(S), len(M)))
    
    # Identity mappings
    for i in range(4):
        dS[i, i] = 1
    
    # Standard deviations
    if S[4] > 0:
        dS[4, 4] = 1/(2*S[4])
        dS[4, 0] = -M[0]/S[4]
    if S[5] > 0:
        dS[5, 5] = 1/(2*S[5])
        dS[5, 2] = -M[2]/S[5]
    if S[6] > 0:
        dS[6, 6] = 1/(2*S[6])
        dS[6, 3] = -M[3]/S[6]
    if S[10] > 0:
        dS[10, 11] = 1/(2*S[10])
        dS[10, 1] = -M[1]/S[10]
    
    # Autocorrelations
    if S[5] > 0:
        dS[7, 7] = 1/S[5]**2
        dS[7, 2] = 2*M[2]*(M[7]-M[5])/(S[5]**4)
        dS[7, 5] = -2*(S[7]/S[5])*dS[5, 5]
    
    # Beta and R-squared
    if S[5] > 0:
        dS[8, 2] = (-M[8]*S[5]**2 - 2*S[5]*dS[5, 2]*(M[10]-M[2]*M[8]))/S[5]**4
        dS[8, 5] = -2*(S[8]/S[5])*dS[5, 5]
        dS[8, 8] = -M[2]/S[5]**2
        dS[8, 10] = 1/S[5]**2
    
    return dS

def compute_newey_west(hy: np.ndarray, M: np.ndarray, N: int, 
                       method: int = 1) -> np.ndarray:
    """Compute Newey-West covariance estimator"""
    Mn = np.tile(M.reshape(-1, 1), (1, N))
    hy_Mn = hy - Mn
    
    # Compute C0
    C0 = (1/N) * (hy_Mn @ hy_Mn.T)
    
    if method == 1:  # Newey-West
        K = int(N/4)
        Sw = C0.copy()
        
        for j in range(1, K+1):
            wjK = (1 - j/(K+1)) * (1/(N-j))
            if j < hy_Mn.shape[1]:
                Cj = wjK * hy_Mn[:, j:] @ hy_Mn[:, :-j].T
                Sw = Sw + (Cj + Cj.T)
    else:
        # Den Haan Levin method (not implemented)
        Sw = C0
    
    return Sw

In [16]:
# ============================================================================
# Learning Model Implementation
# ============================================================================

def est_learning_twoshocks(param_est: np.ndarray, param_fix: np.ndarray, 
                           PD_max: float, method: str = 'tracking',
                           proj_type: int = 1, lags: int = 5, 
                           VCV: Optional[np.ndarray] = None,
                           stat_sel: Optional[np.ndarray] = None,
                           par_select: Optional[np.ndarray] = None,
                           ModelWeightM: int = 0, onlydiag: int = 0,
                           g: Optional[GlobalData] = None) -> Tuple:
    """
    Simulate learning model with two shocks
    """
    if g is None:
        g = GlobalData()
    
    # Combine parameters
    param = np.zeros(5)
    if par_select is not None:
        param[par_select] = param_est
        param[~par_select] = param_fix
    else:
        param = param_est
    
    # Transform parameters back
    param = np.exp(param)
    param[0] = param[0] + 1
    param[3] = param[3] / (1 + param[3])
    param[[1, 2, 4]] = param[[1, 2, 4]] + 1e-3
    
    # Extract parameters
    a, s, sigma, delta, alpha_init = param
    
    # Consumption shock parameters
    s_c = s / 7
    corr_cd = 0.2
    cov_cd = corr_cd * s_c * s
    SIGMA = np.array([[s**2, cov_cd], [cov_cd, s_c**2]])
    
    # Risk-adjusted price growth
    bRE = a**(1-sigma) * np.exp(sigma*(1+sigma)*s_c**2/2 - sigma*cov_cd)
    
    # Check validity
    if delta*bRE > PD_max/(PD_max + g.ratio_proj) or bRE < 1e-3:
        return 1e10, None, None, None, None, None, param
    
    # Draw shocks
    np.random.seed(123)
    n_shocks = g.realizations * (g.N + g.init)
    shocks = np.random.multivariate_normal([0, 0], SIGMA, n_shocks)
    
    # Reshape shocks
    Shocks_Div = np.exp(shocks[:, 0].reshape(g.realizations, g.N + g.init) - s**2/2)
    Shocks_Con = np.exp(shocks[:, 1].reshape(g.realizations, g.N + g.init) - s_c**2/2)
    
    # Initialize arrays
    b = np.zeros((g.realizations, g.N))
    alpha = np.ones(g.realizations) * alpha_init
    b[:, 0] = bRE
    PD = np.zeros((g.realizations, g.N))
    PD[:, 0] = delta * bRE / (1 - delta * b[:, 0])
    b_realiz = np.ones(g.realizations) * bRE
    rb = np.zeros((g.realizations, g.N))
    
    # Simulation loop
    for t in range(1, g.N + g.init):
        if t < g.N:
            # Update learning parameter
            if method.lower() == 'ols':
                alpha = alpha + 1
            elif method.lower() == 'tracking':
                alpha = np.ones(g.realizations) * alpha_init
            elif method.lower() == 'switching_weights':
                nu = 0.1
                do_switch = np.abs(b_realiz - b[:, t-1]) <= nu
                alpha[do_switch] = alpha[do_switch] + 1
                alpha[~do_switch] = alpha_init
            
            # Belief updating
            b[:, t] = b[:, t-1] + (1/alpha) * (b_realiz - b[:, t-1])
            
            # Projection facility
            epsilon = bRE / PD_max
            if proj_type == 1:
                bpos = b[:, t] > (1/delta - g.ratio_proj * epsilon)
                if np.any(bpos):
                    weight = 1 - epsilon / (b[bpos, t] - 1/delta + (g.ratio_proj + 1) * epsilon)
                    b[bpos, t] = weight * (1/delta - epsilon) + (1 - weight) * (1/delta - g.ratio_proj * epsilon)
            
            # Update prices and returns
            PD[:, t] = delta * bRE / (1 - delta * b[:, t])
            rb[:, t] = (delta * a**(-sigma) * np.exp(sigma*(1+sigma)*s_c**2/2))**(-1)
            b_realiz = ((a * Shocks_Div[:, t]) * (a * Shocks_Con[:, t])**(-sigma) * 
                       (1 - delta * b[:, t-1]) / (1 - delta * b[:, t]))
    
    # Compute statistics
    stats_dict = compute_model_statistics(PD, rb, Shocks_Div, Shocks_Con, a, lags, g)
    
    # Compute objective
    obj = 0
    if VCV is not None and stat_sel is not None and g.MOM_DATA is not None:
        WeightM = inv(VCV[np.ix_(stat_sel, stat_sel)])
        if onlydiag:
            WeightM = inv(np.diag(np.diag(VCV[np.ix_(stat_sel, stat_sel)])))
        
        diff = g.MOM_DATA[stat_sel] - stats_dict['mean'][stat_sel]
        obj = diff.T @ WeightM @ diff
        
        if not ModelWeightM:
            T = g.N - 4*lags - 1
            obj = T * obj
    
    return (obj, stats_dict.get('vcv'), stats_dict.get('mean'),
            stats_dict.get('ci'), stats_dict.get('std'),
            stats_dict.get('add_stats'), param)

def compute_model_statistics(PD: np.ndarray, rb: np.ndarray, 
                            Shocks_Div: np.ndarray, Shocks_Con: np.ndarray,
                            a: float, lags: int, g: GlobalData) -> Dict:
    """Compute model statistics from simulation"""
    init = g.init
    
    # Stock returns - ensure compatible shapes
    # PD has shape (realizations, N), Shocks_Div has shape (realizations, N+init)
    n_periods = PD.shape[1] - 1
    rs = ((1 + PD[:, 1:]) / PD[:, :-1]) * (a * Shocks_Div[:, init+1:init+1+n_periods])
    
    # Create indices
    stock_index = np.cumprod(rs, axis=1)
    bond_index = np.cumprod(rb[:, 1:], axis=1)
    
    # Adjust for lags
    if rs.shape[1] <= 4*lags:
        # Not enough data for statistics
        return {
            'mean': np.zeros(12),
            'std': np.zeros(12),
            'ci': (np.zeros(12), np.zeros(12)),
            'vcv': np.eye(12),
            'add_stats': np.zeros(3)
        }
    
    # Means - adjust indices properly
    Mean_rs = np.mean(rs[:, :max(1, rs.shape[1]-4*lags)], axis=1)
    Mean_rb = np.mean(rb[:, 1:max(2, rb.shape[1]-4*lags)], axis=1) if rb.shape[1] > 1 else np.zeros(g.realizations)
    Mean_PD = np.mean(PD[:, 1:max(2, PD.shape[1]-4*lags)], axis=1) if PD.shape[1] > 1 else np.zeros(g.realizations)
    Mean_D = np.mean(a * Shocks_Div[:, init+1:max(init+2, Shocks_Div.shape[1]-4*lags)], axis=1)
    
    # Standard deviations
    sd_rs = np.std(rs[:, :max(1, rs.shape[1]-4*lags)], axis=1)
    sd_PD = np.std(PD[:, 1:max(2, PD.shape[1]-4*lags)], axis=1) if PD.shape[1] > 1 else np.zeros(g.realizations)
    sd_D = np.std(a * Shocks_Div[:, init+1:max(init+2, Shocks_Div.shape[1]-4*lags)], axis=1)
    sd_rb = np.std(rb[:, 1:max(2, rb.shape[1]-4*lags)], axis=1) if rb.shape[1] > 1 else np.zeros(g.realizations)
    
    # Autocorrelations
    autcorr_PD = compute_autocorr(PD[:, 1:max(2, PD.shape[1]-4*lags)])
    autcorr_rs = compute_autocorr(rs[:, :max(1, rs.shape[1]-4*lags)])
    
    # Excess returns and regression
    if stock_index.shape[1] > 4*lags and bond_index.shape[1] > 4*lags:
        n_ret = min(stock_index.shape[1], bond_index.shape[1])
        exc_ret = ((stock_index[:, 4*lags:n_ret] / stock_index[:, :n_ret-4*lags]) /
                   (bond_index[:, 4*lags:n_ret] / bond_index[:, :n_ret-4*lags]))
        betaPD, Rsquare = compute_regression_stats(PD[:, 1:exc_ret.shape[1]+1], exc_ret)
    else:
        betaPD = np.zeros(g.realizations)
        Rsquare = np.zeros(g.realizations)
    
    # Combine statistics
    Distr_STATS = np.column_stack([
        Mean_rs, Mean_rb, Mean_PD, Mean_D,
        sd_rs, sd_PD, sd_D, autcorr_PD,
        betaPD, Rsquare, sd_rb, autcorr_rs
    ])
    
    # Summary statistics
    stats_mean = np.mean(Distr_STATS, axis=0)
    stats_std = np.std(Distr_STATS, axis=0)
    stats_sorted = np.sort(Distr_STATS, axis=0)
    
    CI_low = int((1 - g.CI) * g.realizations)
    CI_high = int(g.CI * g.realizations)
    
    return {
        'mean': stats_mean,
        'std': stats_std,
        'ci': (stats_sorted[CI_low, :], stats_sorted[CI_high, :]),
        'vcv': np.cov(Distr_STATS.T),
        'add_stats': np.array([
            np.mean(np.log(rs[:, :-4*lags])),
            np.mean(np.log(rb[:, init+1:-4*lags])),
            np.mean(PD[:, -4*lags] / PD[:, 0])
        ])
    }

def compute_autocorr(x: np.ndarray) -> np.ndarray:
    """Compute autocorrelation for each row"""
    autocorr = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        if x.shape[1] > 1:
            autocorr[i] = np.corrcoef(x[i, 1:], x[i, :-1])[0, 1]
    return autocorr

def compute_regression_stats(PD: np.ndarray, exc_ret: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Compute regression statistics"""
    betaPD = np.zeros(PD.shape[0])
    Rsquare = np.zeros(PD.shape[0])
    
    for i in range(PD.shape[0]):
        if exc_ret.shape[1] > 0:
            n = min(PD.shape[1], exc_ret.shape[1])
            if n > 1:
                corr = np.corrcoef(PD[i, :n], exc_ret[i, :n])[0, 1]
                Rsquare[i] = corr**2
                betaPD[i] = corr * (np.std(exc_ret[i, :n]) / np.std(PD[i, :n]))
    
    return betaPD, Rsquare


In [13]:
# ============================================================================
# R-squared Computation
# ============================================================================

def compute_rsquare(dS_Sw_dS: np.ndarray) -> pd.DataFrame:
    """
    Compute R-squared statistics
    """
    # Take first 10x10 submatrix
    Sigma = dS_Sw_dS[:10, :10]
    n = len(Sigma)
    
    Rsquared = np.zeros(n)
    
    for i in range(n):
        mask = np.ones(n, dtype=bool)
        mask[i] = False
        
        V_i = Sigma[np.ix_(mask, mask)]
        covi_i = Sigma[mask, i]
        Vi = Sigma[i, i]
        
        if Vi > 0 and np.linalg.det(V_i) != 0:
            Rsquared[i] = 1 - (covi_i.T @ inv(V_i) @ covi_i) / Vi
    
    labels = [
        'E(r^s)', 'E(r^b)', 'E(PD)', 'E(D/d)',
        'sigma_r^s', 'sigma_PD', 'sigma_D/d',
        'autcorr_PD', 'betaPD', 'R^2'
    ]
    
    return pd.DataFrame({'Statistic': labels, 'R-squared': Rsquared})

In [14]:
# ============================================================================
# Results Display
# ============================================================================

def show_results_table(STATS: np.ndarray, STD: np.ndarray, 
                      MOM_DATA: np.ndarray, std_MOM_DATA: np.ndarray,
                      match_stat: np.ndarray) -> pd.DataFrame:
    """Display results table"""
    vlab = [
        'E(r^s)', 'E(r^b)', 'E(PD)', 'E(D/d)',
        'sigma_r^s', 'sigma_PD', 'sigma_D/d', 'autcorr_PD',
        'betaPD', 'R^2', 'sigma_r^b', 'autcorr_r^s'
    ]
    
    # Scaling
    scale_prod = np.array([100, 100, 1, 100, 100, 1, 100, 1, 1, 1, 100, 1])
    scale_sum = np.array([-1, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0])
    
    # Scale data
    STATS_Table = (STATS + scale_sum) * scale_prod
    MOM_DATA_Table = (MOM_DATA + scale_sum) * scale_prod
    STD_Table = STD * scale_prod
    std_MOM_DATA_Table = std_MOM_DATA * scale_prod
    
    # T-statistics
    t_stats = (MOM_DATA_Table - STATS_Table) / std_MOM_DATA_Table
    
    return pd.DataFrame({
        'Statistic': vlab,
        'Model Mean': STATS_Table,
        'Model Std': STD_Table,
        'Data Mean': MOM_DATA_Table,
        'Data Std': std_MOM_DATA_Table,
        't-stat': t_stats,
        'Used': match_stat.astype(int)
    })

In [17]:
# ============================================================================
# Main Execution
# ============================================================================

def main():
    """Main execution function"""
    print("=" * 80)
    print(" STOCK MARKET VOLATILITY AND LEARNING ".center(80))
    print(" Adam, Marcet, Nicolini Model Implementation ".center(80))
    print("=" * 80)
    print()
    
    # Initialize
    g = GlobalData()
    g.realizations = 1000
    g.N = 500
    
    # Load data
    print("📊 Computing weight matrix...")
    dS_Sw_dS, S, N = compute_weight_matrix(lag=5, method=1)
    g.MOM_DATA = S
    g.N = N
    g.dS_Sw_dS = dS_Sw_dS
    
    # Compute R-squared
    print("📈 Computing R-squared statistics...")
    rsquare_df = compute_rsquare(dS_Sw_dS)
    print(rsquare_df)
    print()
    
    # Model configuration
    config = ModelConfig()
    params = Parameters()
    
    # Setup parameters
    param_guess = np.array([params.a, params.s, params.sigma, params.delta, params.alpha])
    par_select = np.array([True, True, False, True, True])
    match_stat = np.ones(len(S), dtype=bool)
    stat_sel = np.arange(len(S))[match_stat]
    
    # Transform parameters
    param = np.array([
        np.log(params.a - 1),
        np.log(params.s - 1e-3),
        np.log(params.sigma - 1e-3),
        np.log(params.delta - 1e-3),
        np.log(params.alpha - 1e-3)
    ])
    
    param_est = param[par_select]
    param_fix = param[~par_select]
    
    # Run simulation
    print("🔄 Running simulation...")
    obj, VCV_model, STATS, CINT, STD, ADD_STATS, param_res = est_learning_twoshocks(
        param_est, param_fix, config.PD_max,
        method=config.method, proj_type=config.proj_type,
        lags=config.lags, VCV=dS_Sw_dS, stat_sel=stat_sel,
        par_select=par_select, ModelWeightM=0, onlydiag=0, g=g
    )
    
    # Display results
    if STATS is not None:
        print("📋 RESULTS")
        BigSigma = dS_Sw_dS
        std_MOM_DATA = np.diag(np.sqrt(BigSigma / (N - 4*config.lags - 1)))
        
        results_df = show_results_table(STATS, STD, g.MOM_DATA, std_MOM_DATA, match_stat)
        print(results_df)
        
        # Test statistics
        dgf = np.sum(match_stat) - np.sum(par_select)
        if dgf > 0:
            print(f"\n🔍 Test Statistics:")
            print(f"   Objective: {obj:.4f}")
            print(f"   DOF: {dgf}")
            p_value = 1 - stats.chi2.cdf(obj, dgf)
            print(f"   p-value: {p_value:.4f}")
    
    print("\n✓ Complete!")
    
    return {
        'config': config,
        'params': param_res,
        'statistics': STATS,
        'confidence_intervals': CINT,
        'std_dev': STD,
        'objective': obj
    }

if __name__ == "__main__":
    results = main()

                      STOCK MARKET VOLATILITY AND LEARNING                      
                  Adam, Marcet, Nicolini Model Implementation                   

📊 Computing weight matrix...
📈 Computing R-squared statistics...
    Statistic  R-squared
0      E(r^s)        0.0
1      E(r^b)        0.0
2       E(PD)        0.0
3      E(D/d)        0.0
4   sigma_r^s        0.0
5    sigma_PD        0.0
6   sigma_D/d        0.0
7  autcorr_PD        0.0
8      betaPD        0.0
9         R^2        0.0

🔄 Running simulation...


LinAlgError: singular matrix